In [1]:
# Imports
# Basics
import pandas as pd 
import numpy as np
import random
from matplotlib import pyplot as plt
%matplotlib inline

# gensim
import gensim

# keras
np.random.seed(13)
from keras.models import Sequential
from keras.layers import (Dense, Embedding, Reshape, Activation, 
                          SimpleRNN, LSTM, Convolution1D, 
                          MaxPooling1D, Dropout, Bidirectional)
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.datasets import imdb, reuters
from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop


# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Using TensorFlow backend.


In [17]:
import pickle
import spacy
import scattertext as st
import re

import spacy
#from spacy import displacy
#from spacy.symbols import amod
from spacy.lang.en.stop_words import STOP_WORDS

from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

import pyLDAvis
import pyLDAvis.sklearn

import pandas as pd
import numpy as np

import swat
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import nltk
#nltk.download()
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

/Users/elena/opt/anaconda3/lib/python3.7/site-packages/nltk/decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""


In [37]:
!pwd

/Users/elena/Desktop/Metis/projects/5_project/Blogger_Boost/code


In [39]:
with open('../../blogger_com_data_330677_7.pkl', 'rb') as picklefile:
    df_toy = pickle.load(picklefile)

In [40]:
df_toy

,date,blogger_id,gender,age,occupation,name,post
0,"19,August,2004",4162441,male,16,Student,Sagittarius,DESTINY... you might not say a...
13,"19,August,2004",4157968,male,16,Student,Pisces,"As I strolled into the mall yesterday,..."
17,"19,August,2004",4278694,female,24,Technology,Virgo,I've never really had a blog bef...
69,"19,August,2004",4261114,female,40,indUnk,Gemini,I love to read novels. I l...
79,"19,August,2004",4215047,male,17,Student,Aquarius,today.. really a very sad day...
...,...,...,...,...,...,...,...
50463676,"12,February,1999",2999514,male,27,Engineering,Pisces,[detail] Propog...
50482996,"01,April,1999",2999514,male,27,Engineering,Pisces,[detail] I actually...
50502316,"05,June,1999",2999514,male,27,Engineering,Pisces,[detail] A tria...
50521636,"16,December,1999",2999514,male,27,Engineering,Pisces,[detail] Points...


In [43]:
corpus = df_toy['post'][0]
corpus

"          DESTINY...       you might not say anything     but i can hear     you have chosen me, your life partner     so have i dear,     so have i dear....      my first dream, my first extreme,     my first love, i was waiting for my DESTINY.     what should i do with myself,     tell me o' my heart     what should i do with myself,     tell me....      should i fly, with this beautiful nature.     or should i play with these winds.     should i try to reach the skies,     or should i pray to the mother earth.     what should i do with myself friends     tell me....      she talked in such a way,     gave me dreams with thousand colours.     like i stand in the middle of island,     and she shows me all the love she has,     my first dream , my first extreme,     my first love, i was waiting for my DESTINY.           --NIL            "

In [42]:
# Load in Corpus using Keras utility
# We'll use some Lovecraft
##!curl -o lovecraft.txt https://raw.githubusercontent.com/urschrei/lovecraft/master/lovecraft.txt

corpus = open("lovecraft.txt").readlines()[0:200]

corpus[6]

'That is not dead which can eternal lie, And with strange aeons death may die.\n'

In [44]:
corpus = [word for word in corpus if word.count(" ") >= 1]
corpus

[' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' '

In [26]:
tokenizer = Tokenizer(filters="""!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',""")
tokenizer.fit_on_texts(corpus)

In [28]:
sequences = tokenizer.texts_to_sequences(corpus)
sequences

[[8],
 [1],
 [6],
 [2],
 [4],
 [9],
 [10],
 [],
 [14],
 [4],
 [15],
 [11],
 [2],
 [],
 [6],
 [5],
 [10],
 [],
 [5],
 [9],
 [10],
 [2],
 [11],
 [4],
 [9],
 [15],
 [],
 [11],
 [1],
 [5],
 [3],
 [],
 [19],
 [11],
 [13],
 [6],
 [1],
 [9],
 [],
 [7],
 [4],
 [12],
 [1],
 [],
 [20],
 [5],
 [3],
 [2],
 [9],
 [1],
 [3],
 [],
 [8],
 [1],
 [5],
 [3],
 [],
 [8],
 [1],
 [5],
 [3],
 [],
 [12],
 [4],
 [3],
 [6],
 [2],
 [],
 [8],
 [3],
 [1],
 [5],
 [14],
 [],
 [12],
 [4],
 [3],
 [6],
 [2],
 [],
 [1],
 [22],
 [2],
 [3],
 [1],
 [14],
 [1],
 [],
 [12],
 [4],
 [3],
 [6],
 [2],
 [],
 [7],
 [13],
 [18],
 [1],
 [],
 [16],
 [5],
 [4],
 [2],
 [4],
 [9],
 [15],
 [],
 [8],
 [1],
 [6],
 [2],
 [4],
 [9],
 [10],
 [],
 [2],
 [1],
 [7],
 [7],
 [],
 [11],
 [1],
 [5],
 [3],
 [2],
 [],
 [2],
 [1],
 [7],
 [7],
 [],
 [12],
 [7],
 [10],
 [],
 [23],
 [1],
 [5],
 [17],
 [2],
 [4],
 [12],
 [17],
 [7],
 [],
 [9],
 [5],
 [2],
 [17],
 [3],
 [1],
 [],
 [20],
 [7],
 [5],
 [10],
 [],
 [16],
 [4],
 [9],
 [8],
 [6],
 [],
 [2],
 [3],


In [22]:
# For simplicity, one "sentence" per line 
corpus = [sentence for sentence in corpus if sentence.count(" ") >= 2]

# Tokenize using Keras
tokenizer = Tokenizer(filters="""!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',""")
tokenizer.fit_on_texts(corpus)

# Convert tokenized sentences to sequence format
sequences = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)

print(corpus[3])
print(sequences[3])

IndexError: list index out of range

In [30]:
nb_samples = sum(len(s) for s in corpus)
nb_samples

342

In [32]:
print(corpus[0])

d


In [10]:
# Setting parameters for our model:

# Vocab size
V = len(tokenizer.word_index) + 1

# Dimension to reduce to (length of word embedding vectors)
dim = 100
window_size = 2

print("vocabulary size: ", V)

vocabulary size:  3316


In [11]:
# Function to generate the inputs and outputs for all windows
def generate_data(sequences, window_size, V):
    maxlen = window_size*2
    # For each line (sentence)
    for words in sequences:
        L = len(words)
        # Choose the target word
        for index, word in enumerate(words):
            # Create the window
            s = index-window_size
            e = index+window_size+1
                    
            in_words = []
            labels = []
            # Create the input/outputs for skipgrams
            for i in range(s, e):
                if i != index and 0 <= i < L:
                    in_words.append([word] )
                    labels.append(words[i])

            x = np.array(in_words,dtype=np.int32)
            y = np_utils.to_categorical(labels, V)
            yield (x, y)
            
# We'll call this later on, from within our train_skipgram_model function.

In [12]:
# Create the Keras model and view it 
skipgram = Sequential()
skipgram.add(Embedding(input_dim=V, input_length=1, embeddings_initializer="glorot_uniform", output_dim=dim))
skipgram.add(Reshape((dim, )))
skipgram.add(Dense(input_dim=dim, units=V, activation='softmax'))

skipgram.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1, 100)            331600    
_________________________________________________________________
reshape_1 (Reshape)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3316)              334916    
Total params: 666,516
Trainable params: 666,516
Non-trainable params: 0
_________________________________________________________________


In [13]:

def train_skipgram_model(skipgram, sequences, window_size, dimension_size=V):
    """
    skipgram: Keras model to train
    sequences: list of lists of integers. 
               sequences[i][j] is the encoding of word j in document i
    window_size: number of words in the window
    dimension_size: integer. Size of the vector space for the word vectors.
    
    Note: this is slow to train. Took an 1hr 40min (no GPU) on 2016 Macbook Pro.
    """
    # Note this cell took 1hr 40min on my machine (no GPU)
    # Compile the Keras Model
    skipgram.compile(loss='categorical_crossentropy', optimizer="adadelta")

    # Fit the Skipgrams
    for iteration in range(10):
        loss = 0.
        for sequence, label in generate_data(sequences, window_size, dimension_size):
            loss += skipgram.train_on_batch(sequence, label)

        print(iteration, loss)

In [14]:
def write_model_weights(skipgram, tokenizer, filename='vectors.txt'):
    # Write the resulting vectors to a text file
    with open(filename ,'w') as f:
        f.write(f"{V-1} {dim}\n")
        vectors = skipgram.get_weights()[0]
        for word, i in tokenizer.word_index.items():
            line = f"{word} " + " ".join([str(num) for num in vectors[i,:]]) + "\n"
            f.write(line)

In [15]:
import os

if not os.path.exists('vectors.txt'):
    train_skipgram_model(skipgram, sequences, window_size, V)
    write_model_weights(skipgram, token)


/Users/elena/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0 105299.83390712738
1 99556.8950150013
2 99391.85418915749
3 99290.92662787437
4 99150.98761761189
5 99028.40557861328
6 98942.17820227146
7 98846.22163951397
8 98756.67071413994
9 98677.5851148367


NameError: name 'token' is not defined

In [ ]:
# Load the vectors into word2vec and see how we did!
w2v = gensim.models.KeyedVectors.load_word2vec_format('./vectors.txt', binary=False)
w2v.most_similar(positive=['white', 'rabbit'])

In [3]:
max_features = 20000
maxlen = 100  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

Loading data...


/Users/elena/opt/anaconda3/lib/python3.7/site-packages/keras/datasets/imdb.py:49: UserWarning: The `nb_words` argument in `load_data` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `load_data` '


17465344/17464789 [==============================] - 2s 0us/step
25000 train sequences
25000 test sequences


In [7]:
y_test

array([0, 1, 1, ..., 0, 0, 0])

In [ ]:
max_features = 20000
maxlen = 100  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print("Pad sequences (samples x time)")
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
y_train = np.array(y_train)
y_test = np.array(y_test)

brnn_model = Sequential()
brnn_model.add(Embedding(max_features, 128, input_length=maxlen))

# Bidirectional LSTM!!!
brnn_model.add(Bidirectional(LSTM(64)))
brnn_model.add(Dropout(0.5))
brnn_model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
brnn_model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
brnn_model.fit(X_train, y_train,
               batch_size=batch_size,
               epochs=4,
               validation_data=[X_test, y_test])

score, acc = brnn_model.evaluate(X_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

In [2]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features)

NameError: name 'max_features' is not defined